In [ ]:
# Import modules

%matplotlib inline

import os
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(10)

import matplotlib.animation as animation

default_dir = os.path.dirname(os.getcwd())
os.chdir(default_dir)

import RLConn as rc
from RLConn import problem_definitions as problems

## Import ground truth modes

In [ ]:
m1_gt = rc.neural_params.m1_gt
m2_gt = rc.neural_params.m2_gt

In [ ]:
plt.figure(figsize=(5.5,5))

plt.scatter(m1_gt, m2_gt, s = 10, color = 'black')
plt.ylim(-45, 45)
plt.xlim(-45, 45)

## Define problem statement params and DQN params

In [ ]:
Gg_noise = np.random.randint(-2, 2, (3,3))
Gs_noise = np.random.randint(-2, 2, (3,3))

Gg_groundtruth = np.array([[0, 8, 5],
               [8, 0, 2],
               [5, 2, 0]])

Gg_noised = Gg_groundtruth + Gg_noise

Gs_groundtruth = np.array([[0, 2, 8],
               [7, 0, 3],
               [7, 7, 0]])

Gs_noised = Gs_groundtruth + Gs_noise

E = np.array([1, 0, 0])

In [ ]:
network_dict_init = {
    
    "gap": Gg_noised,
    "syn": Gs_noised,
    "directionality": E
}


external_params_dict = {

"input_vec" : [0, 0.03, 0],
"ablation_mask" : np.ones(3),
"tf" : 10,
"t_delta" : 0.01,
"cutoff_1" : 400,
"cutoff_2" : 900
    
}

# Feel free to change the params

batchsize = 3
num_epochs = 10000
err_threshold = 10
weight_min = 0
weight_max = 8
plotting_period = 1000 * (3*(3-1)/2)

## Training

In [ ]:
# Train

training_result = rc.network_sim.train_network(network_dict_init, external_params_dict, m1_gt, m2_gt,
                    batchsize = batchsize, num_epochs = num_epochs, err_threshold = err_threshold, 
                    weight_min = weight_min, weight_max = weight_max, plotting_period = plotting_period)

In [ ]:
# Display results
Gg_trained = training_result['Gg_list'][your best index]
Gs_trained = training_result['Gs_list'][your best index]
E = training_result['E']

mean_error, sum_error = rc.utils.compute_score(Gg_trained, Gs_trained, E, 
                    input_vec, external_params_dict['ablation_mask'], 
                    external_params_dict['tf'], external_params_dict['t_delta'], 
                       external_params_dict['cutoff_1'], external_params_dict['cutoff_2'],
                    m1_target = m1_gt,
                    m2_target = m2_gt,
                    plot_result = True,
                    verbose = True)